In [8]:
import subprocess
import json
import os
import shutil

In [20]:
def create_copy_of_project(file_path):
  ignore_patterns = shutil.ignore_patterns('.*')
  return shutil.copytree(file_path, file_path + '-copy', dirs_exist_ok=True, ignore=ignore_patterns)

In [26]:
def measure_test_coverage(project_dir):
  try:
    subprocess.run(['cd ' + project_dir + ' && npx nyc --exclude examples --exclude test --exclude benchmarks --reporter=json-summary npm test'], shell=True, capture_output=True, text=True, check=True)
    with open(project_dir + '/coverage/coverage-summary.json', "r") as coverage_summary: 
      coverage_info = json.load(coverage_summary)
    
    coverage_info_cleansed = dict()
    for module in coverage_info:
      coverage_info_cleansed[module.replace(project_dir, '')] = coverage_info[module]
      
    return coverage_info_cleansed
    
  except subprocess.CalledProcessError as e:
    print(f"An error occurred: {e}")
    return None  

In [25]:
def compute_cyclomatic_complexity(project_dir, code_dir):
  try:
      result = subprocess.run(['npx cyclomatic-complexity ' + project_dir + code_dir + '\'/**/*.[tj]s\'' + ' -j'], shell=True, capture_output=True, text=True, check=True)
      complexity_data = json.loads(result.stdout)
      
      files = complexity_data
      complexity_info = [{'name': file['file'].replace(project_dir, ''), 'cyclomatic': file['complexitySum']} for file in files]
      
      return complexity_info
  
  except subprocess.CalledProcessError as e:
      print(f"An error occurred: {e}")
      return None

In [24]:
def main():
  # Measure test coverage before changes
  # Measure complexity of code files
  # Have LLM refactor file
  # see if tests pass
  # Measure test coverage after changes
  # Measure complexity after changes
  # Provide summary of how values have changed

  project_dir = '/media/lebkuchen/ba3cb2a9-fcaa-4daf-8c11-e1006cc10771/Repos/express'
  code_dir = '/lib'
  project_copy_dir = create_copy_of_project(project_dir)

  coverage_info = measure_test_coverage(project_dir)
  complexity_info = compute_cyclomatic_complexity(project_dir, code_dir)

  for module in coverage_info:
      print(f"Module {module} has a branch coverage of {coverage_info[module]['branches']['pct']}")
  
  print('')
  for info in sorted(complexity_info, key=lambda info: info['cyclomatic'], reverse=True):
      print(f"Module {info['name']} has a cyclomatic complexity of {info['cyclomatic']}")

if __name__ == "__main__":
    main()


Module total has a branch coverage of 97.05
Module /index.js has a branch coverage of 100
Module /lib/application.js has a branch coverage of 100
Module /lib/express.js has a branch coverage of 100
Module /lib/request.js has a branch coverage of 95.31
Module /lib/response.js has a branch coverage of 96.44
Module /lib/utils.js has a branch coverage of 100
Module /lib/view.js has a branch coverage of 95.83
Module /lib/middleware/init.js has a branch coverage of 100
Module /lib/middleware/query.js has a branch coverage of 100
Module /lib/router/index.js has a branch coverage of 97.22
Module /lib/router/layer.js has a branch coverage of 90
Module /lib/router/route.js has a branch coverage of 100

Module /lib/response.js has a cyclomatic complexity of 229
Module /lib/router/index.js has a cyclomatic complexity of 144
Module /lib/application.js has a cyclomatic complexity of 94
Module /lib/request.js has a cyclomatic complexity of 64
Module /lib/utils.js has a cyclomatic complexity of 47
Mod

In [4]:
from openai import OpenAI

f = open('openai-key.txt', "r")
api_key = f.read() 
client = OpenAI(api_key=api_key)

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "Respond only in Haikus"},
    {"role": "user", "content": "Summarize the achievements of Carl Ritter von Ghega"}
  ]
)

In [7]:
print(response.choices[0].message.content)

Ritter von Ghega,  
Tracks that span the mountains high,  
Railways through the land.  

A vision of speed,  
Alpine tunnels carved with care,  
Bridges touch the sky.  

His engineering,  
Inspired the world afar,  
Railways, he did gift.
